# Semestrální projekt z NLA1 2024/2025
# Zadání č. 1

## Task 1:
 Mějme reálné matice C, D, E, F ∈ Rn× n. Ukažte, jak vypočítat reálné matice A, B ∈Rn× n spouzetřemi n×nmaticovými násobeními tak, že (A+iB) = (C+iD)(E+iF) (zde i značí komplexní jednotku). Nápověda: W = (C +D)(E −F).

Řešení tohoto úkolu jsem poslala napsané na papíře.

## Task 2:
Mějme vektory x, y ∈ Rn. Vytvoř algoritmus a naprogramuj jej v programovacím jazyce Python, který sestaví Householderovu matici P ∈ Rn× n tak, že Px je násobek y.

In [53]:
import numpy as np

def householder_xy(x, y):
    """
    Vytvoří Householderovu matici P ∈ R^n×n tak, že Px je násobek vektoru y.
    x, y jsou sloupcové vektory (n×1)
    """

    n = len(x)

    # normovaná verze vektoru y
    y_norm = y / np.linalg.norm(y)

    # vektor, kam chceme x odrazit (má délku jako x, směr jako y)
    target = np.linalg.norm(x) * y_norm

    # směrový vektor v
    v = target - x

    # Householderova matice
    P = np.eye(n) - 2 * (v @ v.T) / (v.T @ v)

    return P


In [54]:
x = np.array([[1], [2], [2], [0]])
y = np.array([[1], [1], [1], [1]])

P = householder_xy(x, y)
Px = P @ x

print("Px:\n", Px)
print("y:\n", y)

# Prověření, že Px = α * y
alpha = Px[0][0] / y[0][0]
print("Koeficient násobku α:\n", alpha)


Px:
 [[1.5]
 [1.5]
 [1.5]
 [1.5]]
y:
 [[1]
 [1]
 [1]
 [1]]
Koeficient násobku α:
 1.5


## Task 3: 
# (a) 
Zvol si libovolné (vhodné) matice A, B, C, D, E a F ∈ R3× 3 a proveď s nimi operaci (A+iB) = (C+iD)(E+iF) s použitím svého řešení 1. příkladu. Správnost výpočtu ověř vhodným kódem v programovacím jazyce Python.
# (b) 
Zvol si libovolné (vhodné) vektory x, y ∈ R4. Ručním výpočtem sestav Householderovu matici P ∈ R4× 4 tak, že Px je násobek y. Správnost svého řešení ověř pomocí kódu ze 2. příkladu.

Řešení těchto úkolů jsem poslala napsané na papíře.


In [55]:
import numpy as np

C = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

D = np.array([
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0]
])

E = np.array([
    [2, 1, 0],
    [0, 2, 1],
    [1, 0, 2]
])

F = np.array([
    [0, 2, 1],
    [1, 0, 2],
    [2, 1, 0]
])

# 1 násobení
X = C @ E
# 2 násobení
Y = D @ F
#3 násobení
W = (C + D) @ (E + F)

A_manual = X - Y
B_manual = W - X - Y

print("A (z 3-násobení):\n", A_manual)
print("B (z 3-násobení):\n", B_manual)

A_direct = C @ E - D @ F
B_direct = C @ F + D @ E

print("\nA (z klasického rozkladu):\n", A_direct)
print("B (z klasického rozkladu):\n", B_direct)

if np.allclose(A_manual, A_direct) and np.allclose(B_manual, B_direct):
    print("\n Obě řešení dávají stejné výsledky pro A i B.")
else:
    print("\n Výsledky A nebo B se liší.")


A (z 3-násobení):
 [[ 1  1 -2]
 [-2  1  1]
 [ 1 -2  1]]
B (z 3-násobení):
 [[0 4 2]
 [2 0 4]
 [4 2 0]]

A (z klasického rozkladu):
 [[ 1  1 -2]
 [-2  1  1]
 [ 1 -2  1]]
B (z klasického rozkladu):
 [[0 4 2]
 [2 0 4]
 [4 2 0]]

 Obě řešení dávají stejné výsledky pro A i B.


## Task 4:
Naimplementujte metodu sdružených gradientů podle níže uvedeného algoritmu. Porovnejte časovnou náročnost této implementace s verzí algoritmu uvedenou na přednáškách, resp. na cvičeních, tzn. pro stejné vstupní parametry změřte časy potřebné pro výpočet. Odhadněte, o kolik násobení matice-vektor, resp. vektor-vektor, se liší obě implementace metody.



In [10]:
import numpy as np

def conjugate_gradients(A, b, x0, epsilon=1e-8, max_it=1000, count_ops = False):
   
    mv_count = 0 # počet násobení matice-vektor
    vv_count = 0 # počet násobení vektorů
    # citac iteraci
    k = 0
    # vektor pocatecniho rezidua a pocatecniho smeru
    x = x0
    d = r = b - A @ x
    mv_count += 1
    
    delta_new = r.T @ r
    vv_count += 1
    delta_old = 0

    # vytvorime pole, do ktereho budeme ukladat normy rezidua v jednotlivych iteracich
    residuals = [np.linalg.norm(r)]

    r0_A_norm = np.sqrt(r.T @ A @ r)
    # pro vypocet ukoncovaci podminky   
    
    mv_count += 1

    while k < max_it and ((np.sqrt(r.T @ A @ r) / r0_A_norm) > epsilon):
        # pomocny vypocet (usetrime 4 operace nasobeni matice-vektor):
        Ad = A @ d
        mv_count += 1
        # vypoctete alpha_k, x_{k+1} and r_{k+1}:
        alpha = delta_new / (d.T @ Ad)
        vv_count += 1
        x = x + alpha * d
        r = r - alpha * Ad
        # vypoctete novou normu rezidua:
        delta_old = delta_new
        delta_new = r.T @ r
        vv_count += 1
        # vypoctete beta_k, d_{k+1}:
        beta = delta_new / delta_old
        d = r + beta * d
        # navysime citac iteraci
        k += 1
        # ulozime reziduum
        residuals.append(np.linalg.norm(r))

    return x, k, residuals, mv_count, vv_count

In [11]:
# ÚKOL: Doplňte chybějící kód ve while cyklu metody sdružených gradientů

def conjugate_gradients_lekce(A, b, x0, epsilon=1e-8, max_it=1000):
    """
    Resi system Ax=b s SPD matici A pomoci metody sdruzenych gradientu.
    A: matice soustavy
    b: vektor prave strany
    x0: pocatecni odhad
    epsilon: relativni presnost
    max_it: maximalni pocet iteraci
    """
    mv_count = 0 # pocet nasobeni matice-vektor
    vv_count = 0 # pocet nasobeni vektoru
    # citac iteraci
    k = 0

    # vektor pocatecniho rezidua a pocatecniho smeru
    x = x0 # x0.copy()
    d = r = b - A @ x
    # d = r.copy()
    mv_count += 1
 
    # vytvorime pole, do ktereho budeme ukladat normy rezidua v jednotlivych iteracich
    residuals = [np.linalg.norm(r)]
 
    # pro vypocet ukoncovaci podminky   
    r0_norm = np.linalg.norm(b - A @ x0)    
    mv_count += 1
       
    while k < max_it and ((np.linalg.norm(r) / r0_norm) > epsilon):
        # pomocny vypocet (usetrime 4 operace nasobeni matice-vektor):
        Ad = A @ d
        mv_count += 1

        #  vypoctete alpha_k, x_{k+1} and r_{k+1}:
        alpha = (r.T @ d) / (d.T @ Ad)
        vv_count += 2
        x = x + alpha * d
        r_new = r - alpha * Ad
        
        # vypoctete beta_k, d_{k+1}:
        beta = (r_new.T @ Ad) / (d.T @ Ad)
        vv_count += 2
        d = r_new - beta * d
        r = r_new
        
        # navysime citac iteraci
        k += 1        
    
        # ulozime reziduum
        residuals.append(np.linalg.norm(r))

    return x, k, residuals, mv_count, vv_count


In [12]:
import numpy as np
import time
# testovaní
n = 100
"""Vytvoří náhodnou SPD matici podle definice: A = D + U + U^T"""
U = np.triu(np.random.rand(n, n), 1)
d = 100 * np.random.rand(n, 1)
A = np.diag(d.flatten()) + U + U.T
b = np.random.rand(n, 1)
x0 = np.zeros((n, 1))
x_np = np.linalg.solve(A, b) 

# Čas a počty operací - projektni verze
start = time.perf_counter()
x1, k1, r1, mv1, vv1 = conjugate_gradients(A, b, x0, 1e-8, 1000)
end = time.perf_counter()
print("Pocet iteraci (projektni verze): {}".format(k1))
print("Cas: {} s".format(end - start))
print("Pocet nasobeni matice-vektor: {}".format(mv1))
print("Pocet nasobeni vektor-vektor: {}".format(vv1))
print("||x1 - x_np|| = {}".format(np.linalg.norm(x1 - x_np)))

# Čas a počty operací - přednášková verze
start = time.perf_counter()
x2, k2, r2, mv2, vv2 = conjugate_gradients_lekce(A, b, x0, 1e-8, 1000)
end = time.perf_counter()
print("Pocet iteraci (lekce): {}".format(k2))
print("Cas: {} s".format(end - start))
print("Pocet nasobeni matice-vektor: {}".format(mv2))
print("Pocet nasobeni vektor-vektor: {}".format(vv2))
print("||x2 - x_np|| = {}".format(np.linalg.norm(x2 - x_np)))



Pocet iteraci (projektni verze): 70
Cas: 0.0028985730023123324 s
Pocet nasobeni matice-vektor: 72
Pocet nasobeni vektor-vektor: 141
||x1 - x_np|| = 1.8532513924788266e-09
Pocet iteraci (lekce): 70
Cas: 0.002134510999894701 s
Pocet nasobeni matice-vektor: 72
Pocet nasobeni vektor-vektor: 280
||x2 - x_np|| = 1.8532512943404247e-09


In [51]:
import numpy as np

A = np.array([
    [4, 1, 1],
    [1, 3, -1],
    [1, -1, 2]
], dtype=float)

b = np.array([
    [1],
    [2],
    [3]
], dtype=float)

x_exact = np.linalg.solve(A, b)
print("\nx \n", x_exact)



x 
 [[-1.]
 [ 2.]
 [ 3.]]
